<a href="https://colab.research.google.com/github/rrfsantos/Large-Language-Model-Agents-MOOC-Fall-2024/blob/main/AI%20Agentic%20Design%20Patterns%20with%20AutoGen/L1_Multi_Agent_Conversation_and_Stand_up_Comedy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r /content/drive/MyDrive/AutoGen/requirements.txt

In [11]:
from dotenv import load_dotenv
import warnings
import os
from autogen import ConversableAgent
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
# Load environment variables from .env file
load_dotenv("/content/drive/MyDrive/AutoGen/env")
llm_config = {"config_list": [{"model": "gpt-3.5-turbo", "api_key" : os.getenv("OPENAI_API_KEY")}]}

## Define an AutoGen agent

In [14]:
agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

[autogen.oai.client: 10-28 15:38:26] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [15]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here's one for you: Why don't scientists trust atoms? Because they make up everything!


In [16]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Of course! Can you remind me which joke you'd like me to repeat for you?


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [17]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

[autogen.oai.client: 10-28 15:39:03] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-28 15:39:03] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [18]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, great to meet you! Glad you're up for some laughs. So, I recently tried to make a belt out of watches, but it was a waist of time. Those things are hard to keep on schedule!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, love that one! It's like trying to organize cats in a ballet class, right?

--------------------------------------------------------------------------------
cathy (to joe):

Exactly, Joe! It's all fun and games until the cats start improvising their own wild dance moves. It's like a furry, chaotic performance art piece!

--------------------------------------------------------------------------------


## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [19]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': "Hey Joe, great to meet you! Glad you're up for some laughs. So, "
             'I recently tried to make a belt out of watches, but it was a '
             'waist of time. Those things are hard to keep on schedule!',
  'role': 'user'},
 {'content': "Haha, love that one! It's like trying to organize cats in a "
             'ballet class, right?',
  'role': 'assistant'},
 {'content': "Exactly, Joe! It's all fun and games until the cats start "
             "improvising their own wild dance moves. It's like a furry, "
             'chaotic performance art piece!',
  'role': 'user'}]


In [20]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 100,
                                                             'cost': 0.000273,
                                                             'prompt_tokens': 246,
                                                             'total_tokens': 346},
                                      'total_cost': 0.000273},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 100,
                                                             'cost': 0.000273,
                                                             'prompt_tokens': 246,
                                                             'total_tokens': 346},
                                      'total_cost': 0.000273}}


In [21]:
pprint.pprint(chat_result.summary)

("Exactly, Joe! It's all fun and games until the cats start improvising their "
 "own wild dance moves. It's like a furry, chaotic performance art piece!")


## Get a better summary of the conversation

In [22]:
chat_result = joe.initiate_chat(
    cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, great to meet you! Glad you're up for some laughs. So, I recently tried to make a belt out of watches, but it was a waist of time. Those things are hard to keep on schedule!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, love that one! It's like trying to organize cats in a ballet class, right?

--------------------------------------------------------------------------------
cathy (to joe):

Exactly, Joe! It's all fun and games until the cats start improvising their own wild dance moves. It's like a furry, chaotic performance art piece!

--------------------------------------------------------------------------------


In [23]:
pprint.pprint(chat_result.summary)

('Joe and Cathy enjoyed sharing jokes and trading humorous anecdotes. They '
 'compared the challenge of organizing watches on a belt to herding cats in a '
 'ballet class, highlighting the chaos and unpredictability of both scenarios.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [24]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

[autogen.oai.client: 10-28 15:40:18] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-28 15:40:18] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [25]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Glad to hear you're up for some laughs. Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, I like that one, Cathy! How about this one: Why did the math book look sad? Because it had too many problems!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, I love it! Sounds like that math book could use some tutoring. Here's one for you: Why don't scientists trust atoms? Because they make up everything!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic! Atoms can be so sneaky. Alright, Cathy, one more before I gotta go: Why did the golfer bring two pairs of pants? In cas

In [26]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was "Why did the golfer bring two pairs of pants? In case he got a hole in one!"

--------------------------------------------------------------------------------
cathy (to joe):

Haha, I gotta hand it to you, that one's a hole in one! Thanks for the good laughs, Joe! I gotta go.

--------------------------------------------------------------------------------
